In [3]:
import sys, os
sys.path.insert(0, os.path.abspath("E:\Coding\my_stuff"))
from my_keys import MufexKeys # type: ignore
import numpy as np
from datetime import datetime

from tests.backtest.strat import RSIRisingFalling
from quantfreedom.enums import CandleBodyType

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.backtest import run_df_backtest, or_backtest

from quantfreedom.exchanges.mufex import Mufex

mufex_main = Mufex(
    api_key=MufexKeys.mainnet_neo_api_key,
    secret_key=MufexKeys.mainnet_neo_secret_key,
    use_test_net=False
)
symbol = 'BTCUSDT'

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="5m",
    candles_to_dl=3000,
)

In [5]:
backtest_settings_tuple = BacktestSettings()

dos_tuple = DynamicOrderSettings(
    account_pct_risk_per_trade=np.array([3]),
    max_trades=np.array([4]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

exchange_settings_tuple = mufex_main.set_and_get_exchange_settings_tuple(
    leverage_mode=LeverageModeType.Isolated,
    position_mode=PositionModeType.HedgeMode,
    symbol=symbol,
)

static_os_tuple = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [6]:
long_strat = RSIRisingFalling(
    long_short="long",
    dos_tuple=dos_tuple,
    rsi_length=np.array([14]),
    rsi_is_below=np.array([40, 60, 80]),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
long_strat.plot_signals(candles=candles)

In [8]:
if __name__ == "__main__":
    backtest_results = run_df_backtest(
        backtest_settings_tuple=backtest_settings_tuple,
        candles=candles,
        exchange_settings_tuple=exchange_settings_tuple,
        static_os_tuple=static_os_tuple,
        strategy=long_strat,
        threads=16,
    )

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 3
Total order settings to test: 32
Total combinations of settings to test: 96
Total candles: 3,000
Total candles to test: 288,000
Error: Error sending result: '<coroutine object multiprocess_backtest at 0x000002202F99B140>'. Reason: 'TypeError("cannot pickle 'coroutine' object")'
Error: Error sending result: '<coroutine object multiprocess_backtest at 0x00000267FCDEB140>'. Reason: 'TypeError("cannot pickle 'coroutine' object")'
Error: Error sending result: '<coroutine object multiprocess_backtest at 0x000002EACFA3B140>'. Reason: 'TypeError("cannot pickle 'coroutine' object")'
Error: Error sending result: '<coroutine object multiprocess_backtest at 0x000001F16660B140>'. Reason: 'TypeError("cannot pickle 'coroutine' object")'
Error: Error sending result: '<coroutine object multiprocess_backtest at 0x0000021B8064B140>'. Reason: 'TypeError("cannot pickle 'coroutine' object")'
Er

NameError: name 'dl_ex_candles' is not defined

In [ ]:
order_records_df = or_backtest(
    backtest_settings_tuple=backtest_settings_tuple,
    candles=candles,
    dos_tuple=dos_tuple,
    exchange_settings_tuple=exchange_settings_tuple,
    static_os_tuple=static_os_tuple,
    strategy=long_strat,
    ind_set_index=2,
    dos_index=0,
    plot_results=False,
    logger_bool=True,
)

In [ ]:
order_records_df